<a href="https://colab.research.google.com/github/kenji0011/CSST101-3B-AY2526/blob/main/Salvahan_AT3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Facts

In [1]:

facts = {
    "Juan": {"HighGrades": True, "Applied": True, "StudentLeader": None},
    "Maria": {"HighGrades": True, "Applied": False, "StudentLeader": None},
    "Carlos": {"HighGrades": False, "Applied": None, "StudentLeader": True},
    "Ana": {"HighGrades": True, "Applied": True, "StudentLeader": True}
}

# Representing rules
def is_eligible(student_facts):
    return student_facts.get("HighGrades", False) and \
           student_facts.get("StudentLeader", False) and \
           student_facts.get("Applied", False)

def gets_scholarship(student_eligible):
    return student_eligible

def gets_financial_support(student_has_scholarship):
    return student_has_scholarship

# store derived facts
derived_facts = {student: {} for student in facts}

print("Facts:", facts)

Facts: {'Juan': {'HighGrades': True, 'Applied': True, 'StudentLeader': None}, 'Maria': {'HighGrades': True, 'Applied': False, 'StudentLeader': None}, 'Carlos': {'HighGrades': False, 'Applied': None, 'StudentLeader': True}, 'Ana': {'HighGrades': True, 'Applied': True, 'StudentLeader': True}}


### Part 1 – Unification



In [2]:
# Unification
variable = 'x'
constant = 'Ana'
substitution_set = {variable: constant}

print(f"Unifying Eligible({variable}) with Eligible({constant}).")
print(f"Substitution set: {substitution_set}")

Unifying Eligible(x) with Eligible(Ana).
Substitution set: {'x': 'Ana'}


### Part 2 – Forward Chaining



In [3]:
# Forward Chaining
print("Forward Chaining:")

for student, student_facts in facts.items():
    print(f"\nProcessing {student}:")

    eligible = is_eligible(student_facts)
    if eligible:
        derived_facts[student]["Eligible"] = True
        print(f"  Derived: {student} is Eligible")


        scholarship = gets_scholarship(derived_facts[student].get("Eligible", False))
        if scholarship:
            derived_facts[student]["Scholarship"] = True
            print(f"  Derived: {student} gets Scholarship")

            financial_support = gets_financial_support(derived_facts[student].get("Scholarship", False))
            if financial_support:
                derived_facts[student]["FinancialSupport"] = True
                print(f"  Derived: {student} gets FinancialSupport")
    else:
        print(f"  {student} is not Eligible based on Rule 1 conditions.")

print("\nDerived Facts after Forward Chaining:", derived_facts)

# Explain why Maria and Carlos are not eligible
print("\nExplanation for Maria and Carlos:")
print("Maria is not eligible because she did not apply ('Applied' is False). Rule 1 requires HighGrades, StudentLeader, AND Applied.")
print("Carlos is not eligible because he does not have HighGrades ('HighGrades' is False). Rule 1 requires HighGrades, StudentLeader, AND Applied.")

Forward Chaining:

Processing Juan:
  Juan is not Eligible based on Rule 1 conditions.

Processing Maria:
  Maria is not Eligible based on Rule 1 conditions.

Processing Carlos:
  Carlos is not Eligible based on Rule 1 conditions.

Processing Ana:
  Derived: Ana is Eligible
  Derived: Ana gets Scholarship
  Derived: Ana gets FinancialSupport

Derived Facts after Forward Chaining: {'Juan': {}, 'Maria': {}, 'Carlos': {}, 'Ana': {'Eligible': True, 'Scholarship': True, 'FinancialSupport': True}}

Explanation for Maria and Carlos:
Maria is not eligible because she did not apply ('Applied' is False). Rule 1 requires HighGrades, StudentLeader, AND Applied.
Carlos is not eligible because he does not have HighGrades ('HighGrades' is False). Rule 1 requires HighGrades, StudentLeader, AND Applied.


# Explain why Maria and Carlos are not eligible.

Maria is not eligible because she did not apply (Applied is False). Carlos is not eligible because he does not have HighGrades (HighGrades is False)

### Part 3 – Backward Chaining



In [15]:
# Backward Chaining
print("\nBackward Chaining:")

def backward_chain(goal, student, current_facts, all_facts, derived):
    print(f"  Checking goal: {goal}({student})")

    # 1. Check facts
    if goal in current_facts.get(student, {}):
        print(f"    Goal {goal}({student}) is a known fact.")
        return current_facts[student][goal]
    if goal in all_facts.get(student, {}):
         if all_facts[student][goal] is True:
            print(f"    Goal {goal}({student}) is a known fact.")
            derived[student][goal] = True
            return True
         elif all_facts[student][goal] is False:
             print(f"    Goal {goal}({student}) is a known fact (False).")
             return False

    # 2. Apply rule
    if goal == "Scholarship":
        print(f"    Applying rule: Eligible({student}) -> Scholarship({student})")
        # Check subgoal: Eligible(student)
        return backward_chain("Eligible", student, current_facts, all_facts, derived)
    elif goal == "FinancialSupport":
        print(f"    Applying rule: Scholarship({student}) -> FinancialSupport({student})")
        # Check subgoal: Scholarship(student)
        return backward_chain("Scholarship", student, current_facts, all_facts, derived)
    elif goal == "Eligible":
        print(f"    Applying rule: (HighGrades({student}) ∧ Leader({student}) ∧ Applied({student})) -> Eligible({student})")
        # Check subgoals: HighGrades(student), Leader(student), Applied(student)
        subgoals = ["HighGrades", "StudentLeader", "Applied"]
        all_subgoals_met = True
        for sub in subgoals:
            # Check if the subgoal is a fact or can be derived
            if sub in all_facts.get(student, {}):
                if all_facts[student][sub] is False:
                    print(f"      Subgoal {sub}({student}) is false based on facts.")
                    all_subgoals_met = False
                    break
                elif all_facts[student][sub] is True:
                     print(f"      Subgoal {sub}({student}) is true based on facts.")
                else: # Handle None case for unknown facts
                    print(f"      Subgoal {sub}({student}) is unknown based on facts.")
                    all_subgoals_met = False # Cannot prove if unknown
                    break
            else:
                 print(f"      Subgoal {sub}({student}) is not a known fact.")
                 all_subgoals_met = False # Cannot prove if not a fact
                 break


        if all_subgoals_met:
            print(f"    All subgoals for Eligible({student}) met.")
            derived[student]["Eligible"] = True
            return True
        else:
            print(f"    Not all subgoals for Eligible({student}) met.")
            return False

    # If no rule applies and not a fact
    print(f"    Goal {goal}({student}) cannot be proven.")
    return False

bc_derived_facts = {student: {} for student in facts}

# Goal 1: Scholarship(Maria)
print("\nGoal 1: Scholarship(Maria)")
goal1_result = backward_chain("Scholarship", "Maria", bc_derived_facts, facts, bc_derived_facts)
print("----------------------------------------")
print(f"Conclusion: Scholarship(Maria) is {goal1_result}")
print("----------------------------------------")

# Goal 2: FinancialSupport(Ana)
print("\nGoal 2: FinancialSupport(Ana)")
goal2_result = backward_chain("FinancialSupport", "Ana", bc_derived_facts, facts, bc_derived_facts)
print("----------------------------------------")
print(f"Conclusion: FinancialSupport(Ana) is {goal2_result}")
print("----------------------------------------")

print("Derived Facts after Backward Chaining (for goals checked):", bc_derived_facts)



Backward Chaining:

Goal 1: Scholarship(Maria)
  Checking goal: Scholarship(Maria)
    Applying rule: Eligible(Maria) -> Scholarship(Maria)
  Checking goal: Eligible(Maria)
    Applying rule: (HighGrades(Maria) ∧ Leader(Maria) ∧ Applied(Maria)) -> Eligible(Maria)
      Subgoal HighGrades(Maria) is true based on facts.
      Subgoal StudentLeader(Maria) is unknown based on facts.
    Not all subgoals for Eligible(Maria) met.
----------------------------------------
Conclusion: Scholarship(Maria) is False
----------------------------------------

Goal 2: FinancialSupport(Ana)
  Checking goal: FinancialSupport(Ana)
    Applying rule: Scholarship(Ana) -> FinancialSupport(Ana)
  Checking goal: Scholarship(Ana)
    Applying rule: Eligible(Ana) -> Scholarship(Ana)
  Checking goal: Eligible(Ana)
    Applying rule: (HighGrades(Ana) ∧ Leader(Ana) ∧ Applied(Ana)) -> Eligible(Ana)
      Subgoal HighGrades(Ana) is true based on facts.
      Subgoal StudentLeader(Ana) is true based on facts.
      

### Part 4 – Validity Check



In [16]:
import pandas as pd

data = {
    'HighGrades': [True, True, True, True, False, False, False, False],
    'Leader': [True, True, False, False, True, True, False, False],
    'Applied': [True, False, True, False, True, False, True, False],
    'HighGrades ∧ Leader ∧ Applied': [True, False, False, False, False, False, False, False],
    'Eligible (based on Rule 1)': [True, False, False, False, False, False, False, False], # Based on Rule 1 premise
    'Rule (Premise → Eligible)': [True, True, True, True, True, True, True, True] # (P and Q and R) -> S is True unless (P and Q and R) is True and S is False
}

truth_table_df = pd.DataFrame(data)
print("Truth Table for (HighGrades ∧ Leader ∧ Applied) → Eligible:")
display(truth_table_df)

print("\nTesting each student against the rule:")

def test_rule_for_student(student_name, student_facts):
    high_grades = student_facts.get("HighGrades", False)
    leader = student_facts.get("StudentLeader", False)
    applied = student_facts.get("Applied", False)
    eligible_fact = derived_facts.get(student_name, {}).get("Eligible", False)

    premise = high_grades and leader and applied
    implication = not premise or eligible_fact # Equivalent to premise -> eligible_fact

    print(f"\n{student_name}:")
    print(f"  HighGrades: {high_grades}, Leader: {leader}, Applied: {applied}")
    print(f"  Premise (HighGrades ∧ Leader ∧ Applied): {premise}")
    print(f"  Eligible (derived/fact): {eligible_fact}")
    print(f"  Rule (Premise → Eligible) holds: {implication}")
    if premise and not eligible_fact:
        print("  -> This case would violate the rule if Eligible was not derived as True.")


test_rule_for_student("Juan", facts["Juan"])
test_rule_for_student("Maria", facts["Maria"])
test_rule_for_student("Carlos", facts["Carlos"])
test_rule_for_student("Ana", facts["Ana"])

Truth Table for (HighGrades ∧ Leader ∧ Applied) → Eligible:


,HighGrades,Leader,Applied,HighGrades ∧ Leader ∧ Applied,Eligible (based on Rule 1),Rule (Premise → Eligible)
0,True,True,True,True,True,True
1,True,True,False,False,False,True
2,True,False,True,False,False,True
3,True,False,False,False,False,True
4,False,True,True,False,False,True
5,False,True,False,False,False,True
6,False,False,True,False,False,True
7,False,False,False,False,False,True



Testing each student against the rule:

Juan:
  HighGrades: True, Leader: None, Applied: True
  Premise (HighGrades ∧ Leader ∧ Applied): None
  Eligible (derived/fact): False
  Rule (Premise → Eligible) holds: True

Maria:
  HighGrades: True, Leader: None, Applied: False
  Premise (HighGrades ∧ Leader ∧ Applied): None
  Eligible (derived/fact): False
  Rule (Premise → Eligible) holds: True

Carlos:
  HighGrades: False, Leader: True, Applied: None
  Premise (HighGrades ∧ Leader ∧ Applied): False
  Eligible (derived/fact): False
  Rule (Premise → Eligible) holds: True

Ana:
  HighGrades: True, Leader: True, Applied: True
  Premise (HighGrades ∧ Leader ∧ Applied): True
  Eligible (derived/fact): True
  Rule (Premise → Eligible) holds: True


### Part 5 - Reflection

Logical reasoning is essential in AI because it enables machines to draw valid conclusions
from structured knowledge, mimicking human decision-making. In real life, it ensures fairness
and consistency like awarding scholarships based on transparent, rule-based criteria.
Without logic, AI systems would make arbitrary or biased decisions. Additionally, logical frameworks
allow debugging, verification, and scalability in complex systems from legal judgments to medical diagnoses.